In [1]:
#import orginal file
from google.colab import files
uploaded=files.upload() 
#uploading data

Saving vending_machine_sales.csv to vending_machine_sales.csv


In [2]:
import pandas as pd
#importing data

df=pd.read_csv('vending_machine_sales.csv')

df

,Status,ID,Location,Machine,Product,Category,Transaction,TransDate,Type,RCoil,RPrice,RQty,MCoil,MPrice,MQty,LineTotal,TransTotal,Date
0,Processed,VJ300320611,Brunswick Sq Mall,BSQ Mall x1366 - ATT,Red Bull - Energy Drink - Sugar Free,Carbonated,14515778905,"Saturday, January 1, 2022",Credit,148,3.5,1,148,3.5,1,3.5,3.5,1/1/2022
1,Processed,VJ300320611,Brunswick Sq Mall,BSQ Mall x1366 - ATT,Red Bull - Energy Drink - Sugar Free,Carbonated,14516018629,"Saturday, January 1, 2022",Credit,148,3.5,1,148,3.5,1,3.5,5.0,1/1/2022
2,Processed,VJ300320611,Brunswick Sq Mall,BSQ Mall x1366 - ATT,Takis - Hot Chilli Pepper & Lime,Food,14516018629,"Saturday, January 1, 2022",Credit,123,1.5,1,123,1.5,1,1.5,5.0,1/1/2022
3,Processed,VJ300320611,Brunswick Sq Mall,BSQ Mall x1366 - ATT,Takis - Hot Chilli Pepper & Lime,Food,14516020373,"Saturday, January 1, 2022",Credit,123,1.5,1,123,1.5,1,1.5,1.5,1/1/2022
4,Processed,VJ300320611,Brunswick Sq Mall,BSQ Mall x1366 - ATT,Red Bull - Energy Drink - Sugar Free,Carbonated,14516021756,"Saturday, January 1, 2022",Credit,148,3.5,1,148,3.5,1,3.5,3.5,1/1/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6440,Processed,VJ300320692,EB Public Library,EB Public Library x1380,Lindens - Chocolate Chippers,Food,15603201222,"Wednesday, August 31, 2022",Credit,122,2.0,1,122,2.0,1,2.0,6.0,8/31/2022
6441,Processed,VJ300320692,EB Public Library,EB Public Library x1380,Wonderful Pistachios - Variety,Food,15603201222,"Wednesday, August 31, 2022",Credit,131,2.0,1,131,2.0,1,2.0,6.0,8/31/2022
6442,Processed,VJ300320692,EB Public Library,EB Public Library x1380,Hungry Buddha - Chocolate Chip,Food,15603201222,"Wednesday, August 31, 2022",Credit,137,2.0,1,137,2.0,1,2.0,6.0,8/31/2022
6443,Processed,VJ300320609,GuttenPlans,GuttenPlans x1367,Snapple Tea - Lemon,Non Carbonated,15603853105,"Wednesday, August 31, 2022",Credit,145,2.5,1,145,2.5,1,2.5,2.5,8/31/2022


In [3]:
df=df.dropna()
df=df.dropna(axis=0)
# Reset index after drop
df=df.dropna().reset_index(drop=True)

In [4]:
#ABC Codding
x=0
dfabc=df.groupby('Product')['RQty'].sum().reset_index(name ='Total Amount')

TPS=dfabc['Total Amount'].sum()

#dfTPS=DataFrame
dfabc["TPS"]=TPS
dfabc["ABC%"]=(dfabc['Total Amount']/dfabc['TPS'])*100
number_rows_abc=len(dfabc)

dfabc.sort_values(by=['ABC%'])
import numpy as np
conditions = [
    (dfabc['ABC%'] >= 2),#60%
    (dfabc['ABC%'] >= 1) & (dfabc['ABC%'] < 2),#61-75
    (dfabc['ABC%'] < 1)#rest
]

values = ['A','B','C']

dfabc['abc']=np.select(conditions,values)

import numpy as np
conditions1 = [
    (dfabc['abc'] == 'A'),
    (dfabc['abc'] =='B'),
    (dfabc['abc'] == 'C')
]

values1 = [50,25,10]

dfabc['Initial_stock']=np.select(conditions1,values1)
dfabc['Stock_to_BE']=np.select(conditions1,values1)

In [5]:
#order Automation part #1 input
from google.colab import files
uploaded=files.upload()

Saving Book13.csv to Book13.csv


In [7]:
#order Automation part #2 process
#read book 13
import pandas as pd

dfauto=pd.read_csv('Book13.csv')


dfautodate=dfauto['Date']
x=pd.unique(dfauto['Date'])#number of days its checking
dfauto['Dates'] = np.arange(len(dfauto))
dfauto["Dates"] = dfauto["Dates"] + 1 #adding index of day one to nth date

dfautoproductRQty = {}
occur={}
rowsoccur={}
r=len(x)

for i in range(r): #making occurances per day
 dfautoproductRQty[i]=pd.DataFrame()
 dfautoproductRQty[i]=dfauto[(dfauto.RQty >0) & (dfauto.Date==x[i])]
 occur[i] = dfautoproductRQty[i].groupby(['Product']).size()
 rowsoccur[i] = len(occur[i].axes[0])


for i in range(162): #162 number of products we have
  for p in range(r): #number of days in the given data sheet
    for z in range(rowsoccur[p]):
     if dfabc['Product'][i]==occur[p].index[z]:
      dfabc['Stock_to_BE'][i]=dfabc['Stock_to_BE'][i]-occur[p][z]


# checking if anything is out of stock?
for i in range(162):
  if dfabc['abc'][i]=='A' and dfabc['Stock_to_BE'][i]<20:
    print("order 50 more of..........",dfabc['Product'][i] )

  if dfabc['abc'][i]=='B' and dfabc['Stock_to_BE'][i]<15:
   print("order 30 more of..........",dfabc['Product'][i] )

  if dfabc['abc'][i]=='C' and dfabc['Stock_to_BE'][i]<5:
   print("order 20 more of..........",dfabc['Product'][i] )

#


order 20 more of.......... Red Bull - Energy Drink - Sugar Free
order 20 more of.......... Spindrift - Sparkling Water  Lime


<ipython-input-7-5f52a563d1d3>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfabc['Stock_to_BE'][i]=dfabc['Stock_to_BE'][i]-occur[p][z]
